In [1]:
from manim import *
import numpy as np
from itertools import combinations
import random 

config.media_width = "75%"
config.verbosity = "WARNING"

Manim Community v0.17.3

In [14]:
%%manim -qm ThreeDots

class ThreeDots(ThreeDScene):
    
    def add_objects(self, objects):
        for obj in objects:
            self.add(obj)
    
    def remove_objects(self):
        removed_objects = self.mobjects.copy()
        for arg in self.mobjects:
            self.remove(arg)
            
        return removed_objects
            
    def play_text(self, text='', wait_time=3):
        removed_objects = self.remove_objects()
        
        # Set the size and position of the frame
        self.set_camera_orientation(phi=0 * DEGREES, theta=-90 * DEGREES)
               
        # Create and display the text
        text = Text(text, font_size=50)
        self.play(FadeIn(text))

        self.wait(wait_time) # wait for 5 seconds before ending the scene
        
        self.play(FadeOut(text))
        
        self.set_camera_orientation(phi=75 * DEGREES, theta=60 * DEGREES)
        
        self.add_objects(removed_objects)
        
    
    def construct(self):
        axes = ThreeDAxes()
        
        self.renderer.camera.light_source.move_to(3*IN) # changes the source of the light
        self.set_camera_orientation(phi=75 * DEGREES, theta=60 * DEGREES)
        
        self.add(axes)
        
        # Generate two clusters of random points in 3D space
        np.random.seed(0)
        cluster1 = np.random.rand(10, 3)
        cluster2 = np.random.rand(10, 3) + np.array([2, 0, 0])
        
        # Create Mobjects for the two clusters of dots
        cluster1_dots = VGroup(*[Dot3D(np.array(coords), color=BLUE) for coords in cluster1])
        cluster2_dots = VGroup(*[Dot3D(np.array(coords), color=RED) for coords in cluster2])
        
#         colors = [GREEN, PURPLE, PINK]
#         cluster_dots_arr = []
#         for idx, color in enumerate(colors): 
#             rand_x = random.randint(0, 10)
#             rand_y = random.randint(0, 10)
#             rand_z = random.randint(0, 10)
#             cluster = np.random.rand(10, 3) + np.array([rand_x, rand_y, rand_z])
#             cluster_dots = VGroup(*[Dot3D(np.array(coords), color=color) for coords in cluster])
#             cluster_dots_arr.append(cluster_dots)
        
#         print('len ', len(cluster_dots_arr))
#         print(cluster_dots_arr)
        # Calculate the centroid of each cluster
        centroid1 = np.mean(cluster1, axis=0)
        centroid2 = np.mean(cluster2, axis=0)
        
        # Create Mobjects for the centroids
        centroid1_dot = Dot3D(np.array(centroid1), color=YELLOW, radius=0.1)
        centroid2_dot = Dot3D(np.array(centroid2), color=YELLOW, radius=0.1)
        
        # Calculate vector 'd' connecting centroids
        d = centroid2 - centroid1
        d = d/np.linalg.norm(d)

        # calculate matrix that projects into 2D orthogonal complement space
        def calculate_tranformation_matrix(d):
            new_basis_mat = np.eye(d.shape[0])
            new_basis_mat[:, 0] = d
            U, _ = np.linalg.qr(new_basis_mat)
            S = np.eye(U.shape[0])
            S[0, 0] = 0
            
            proj_mat = U @ S @ U.T
            
            return proj_mat
        
        # Collapse 3D points to 2D orthogonal complement space
        def collapse_to_2d(points, proj_mat):
            new_points = [proj_mat @ point for point in points]
            
            return new_points
        
        # Normalize collapsed 2D embedding to unit circle
        def project_to_2D_unit_circle(embeds):    
            # Find the normal vector of the plane
            u, s, vh = np.linalg.svd(embeds)
            normal = vh[-1]    # the last eigenvector is the one that doesn't vary, i.e. 
                               # normal to the plane where the points lie 

            # Project the points onto the plane
            points_proj = embeds - (embeds @ np.outer(normal, normal))  # embeds minus projection of embeds onto normal
                                                                        # would give us embeds that lie on the 2D plane,
                                                                        # which is perpendicular to the normal 

            # normalize to unit circle 
            points_norm = np.linalg.norm(points_proj, axis=1)
            points_proj /= points_norm[:, np.newaxis]         # using broadcasting to normalize 

            return points_proj

        
        proj_mat = calculate_tranformation_matrix(d)
        
        projected_cluster1 = collapse_to_2d(cluster1, proj_mat)
        projected_cluster2 = collapse_to_2d(cluster2, proj_mat)
        centroid1_proj = collapse_to_2d([centroid1], proj_mat)
        centroid2_proj = collapse_to_2d([centroid2], proj_mat)
        
        normalized_cluster1  = project_to_2D_unit_circle(projected_cluster1)
        normalized_cluster2  = project_to_2D_unit_circle(projected_cluster2)
        normalized_centroid1 = project_to_2D_unit_circle(centroid1_proj)
        normalized_centroid2 = project_to_2D_unit_circle(centroid2_proj)
        
        projected_cluster1_dots = VGroup(*[Dot3D(np.array(coords), color=BLUE) for coords in projected_cluster1])
        projected_cluster2_dots = VGroup(*[Dot3D(np.array(coords), color=RED) for coords in projected_cluster2])
        centroid1_dot_proj = Dot3D(np.array(centroid1_proj), color=YELLOW, radius=0.1)
        centroid2_dot_proj =  Dot3D(np.array(centroid2_proj), color=YELLOW, radius=0.1)
        
        normalized_cluster1_dots = VGroup(*[Dot3D(np.array(coords), color=BLUE) for coords in normalized_cluster1])
        normalized_cluster2_dots = VGroup(*[Dot3D(np.array(coords), color=RED) for coords in normalized_cluster2])
        normalized_centroid1_dot = Dot3D(np.array(normalized_centroid1), color=YELLOW, radius=0.1)
        normalized_centroid2_dot = Dot3D(np.array(normalized_centroid2), color=YELLOW, radius=0.1)

        d_vector = Arrow3D(centroid1, centroid2, color=WHITE)
        
        ##################################
        # ANIMATION SECTION
        ##################################
        
        self.play_text('Visualizing network embeddings in 3D')
        # Display original clusters
        self.play(
            Create(cluster1_dots),
            Create(cluster2_dots),
#             *[Create(cluster_dot) for cluster_dot in cluster_dots_arr]
        )
        self.wait()
        
        self.play_text('Calculating centroids of red and blue clusters')
        # Show centroids and vector 'd'
        self.play(
            Create(centroid1_dot),
            Create(centroid2_dot)
        )
        self.wait()
        
        self.play_text("Obtain vector connecting the 2 centroids")
        self.play(
            Create(d_vector),
        )        
        self.wait() # wait for 2 seconds
        
        self.play_text("Collapse the embeddings along this vector")
        self.play(
#             FadeOut(cluster1_dots),
            Transform(cluster1_dots, projected_cluster1_dots),
            FadeOut(d_vector),
#             FadeOut(cluster2_dots),
            Transform(cluster2_dots, projected_cluster2_dots),
#             FadeOut(centroid1_dot),
            Transform(centroid1_dot, centroid1_dot_proj),
#             FadeOut(centroid2_dot),
            Transform(centroid2_dot, centroid2_dot_proj)
        )
        self.wait(3)
        
        
        self.play_text("Normalize embeddings along unit circle")
        self.play(
            FadeOut(cluster1_dots),
            Transform(projected_cluster1_dots, normalized_cluster1_dots),
            FadeOut(cluster2_dots),
            Transform(projected_cluster2_dots, normalized_cluster2_dots),
            FadeOut(centroid1_dot),
            Transform(centroid1_dot_proj, normalized_centroid1_dot),
            FadeOut(centroid2_dot),
            Transform(centroid2_dot_proj, normalized_centroid2_dot)
        )
        self.wait(5)
        
        